In [1]:
!pip install skfeature-chappers
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from collections import Counter
from sklearn.utils import resample
from skfeature.function.similarity_based import fisher_score
import numpy as np
import pandas as pd

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.3/66.3 kB 1.6 MB/s eta 0:00:00a 0:00:01


/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
import random
import math,time,sys, os
from matplotlib import pyplot
from datetime import datetime

In [3]:
## -----------------Reading dataset files------------------------
train_data=pd.read_csv('/kaggle/input/labels/Combined_intra/Combined_intra/train.csv', index_col=False, header = None)
y_train = train_data[train_data.columns[-1]]
(noTrainRows, _) = train_data.shape

#Converting dataseries into pandas dataframe
y_train = y_train.to_frame()
dataframe_train = train_data.drop(train_data.columns[[-1]], axis=1)

test_data=pd.read_csv('/kaggle/input/labels/Combined_intra/Combined_intra/test.csv', index_col=False, header = None)
y_test = test_data[test_data.columns[-1]]

#Converting dataseries in pandas dataframe
y_test = y_test.to_frame()
dataframe_test = test_data.drop(test_data.columns[[-1]], axis=1)
print('Train and Test datasize : ', dataframe_train.shape, dataframe_test.shape)

Train and Test datasize :  (9152, 4352) (518, 4352)


In [4]:
coln=dataframe_train.columns
coln1=y_train.columns

In [5]:
dataframe=dataframe_train.append(dataframe_test,ignore_index=True)
y=y_train.append(y_test,ignore_index=True)
print('Combined size : ',dataframe.shape)

Combined size :  (9670, 4352)


/tmp/ipykernel_32/3658989962.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dataframe=dataframe_train.append(dataframe_test,ignore_index=True)
/tmp/ipykernel_32/3658989962.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  y=y_train.append(y_test,ignore_index=True)


In [6]:
##-----------------Removing columns contains all '0' -------------------
cols=[]
for i in range(len(dataframe.columns)):
    col1=dataframe[dataframe.columns[i]].to_numpy()
    if(np.sum(col1)==0):
        cols.append(dataframe.columns[i])

dataframe = dataframe.drop(columns=cols)
print('Size after removing unnecessary columns :', dataframe.shape)

Size after removing unnecessary columns : (9670, 4005)


In [7]:
df=dataframe.copy()
df['labels']=y
print('After adding labels : ', df.shape)
df = df.replace(np.nan, 0)
df.head()

After adding labels :  (9670, 4006)


,4,5,8,10,13,14,17,27,30,32,...,4343,4344,4345,4346,4347,4348,4349,4350,4351,labels
0,0.018918,0.337158,0.181039,0.000000,0.025402,0.002171,0.279726,0.000000,0.000000,0.001456,...,0.030611,0.018019,0.006840,0.080258,0.015533,0.094722,0.006184,0.069376,0.020940,1
1,0.011344,0.335309,0.144276,0.000000,0.031439,0.004621,0.274816,0.000000,0.000000,0.006705,...,0.046062,0.030992,0.124557,0.181974,0.108694,0.152583,0.006861,0.180436,0.158914,1
2,0.048687,0.258341,0.171693,0.000000,0.011601,0.000912,0.193210,0.000000,0.000169,0.003597,...,0.006135,0.000292,0.011041,0.012258,0.003963,0.027631,0.000000,0.007349,0.007387,1
3,0.017682,0.245461,0.232187,0.000000,0.014165,0.000190,0.209655,0.000179,0.000000,0.002153,...,0.020462,0.006484,0.017934,0.065904,0.018032,0.074175,0.000919,0.055338,0.013800,1
4,0.051619,0.227642,0.255673,0.000009,0.009589,0.000485,0.163954,0.000366,0.000195,0.012668,...,0.015641,0.005714,0.006132,0.055744,0.009117,0.062275,0.001066,0.047317,0.009133,1


In [8]:
x=df.iloc[:,:-1]
x.head()

,4,5,8,10,13,14,17,27,30,32,...,4342,4343,4344,4345,4346,4347,4348,4349,4350,4351
0,0.018918,0.337158,0.181039,0.000000,0.025402,0.002171,0.279726,0.000000,0.000000,0.001456,...,0.141093,0.030611,0.018019,0.006840,0.080258,0.015533,0.094722,0.006184,0.069376,0.020940
1,0.011344,0.335309,0.144276,0.000000,0.031439,0.004621,0.274816,0.000000,0.000000,0.006705,...,0.158972,0.046062,0.030992,0.124557,0.181974,0.108694,0.152583,0.006861,0.180436,0.158914
2,0.048687,0.258341,0.171693,0.000000,0.011601,0.000912,0.193210,0.000000,0.000169,0.003597,...,0.069672,0.006135,0.000292,0.011041,0.012258,0.003963,0.027631,0.000000,0.007349,0.007387
3,0.017682,0.245461,0.232187,0.000000,0.014165,0.000190,0.209655,0.000179,0.000000,0.002153,...,0.099915,0.020462,0.006484,0.017934,0.065904,0.018032,0.074175,0.000919,0.055338,0.013800
4,0.051619,0.227642,0.255673,0.000009,0.009589,0.000485,0.163954,0.000366,0.000195,0.012668,...,0.072792,0.015641,0.005714,0.006132,0.055744,0.009117,0.062275,0.001066,0.047317,0.009133


In [9]:
ln=df.shape[1]
str=df.columns[ln-1]
print(ln, str)

4006 labels


In [10]:
print(x.shape)
dataset=x.to_numpy()

(9670, 4005)


In [11]:
def sort_agents(agents, obj,train_X,val_X,train_Y,val_Y):
    # sort the agents according to fitness
    #train_X, val_X, train_Y, val_Y = data.train_X, data.val_X, data.train_Y, data.val_Y
    (obj_function, weight_acc) = obj

    # if there is only one agent
    if len(agents.shape) == 1:
        num_agents = 1
        fitness = obj_function(agents, train_X, val_X, train_Y, val_Y, weight_acc)
        return agents, fitness

    # for multiple agents
    else:
        num_agents = agents.shape[0]
        fitness = np.zeros(num_agents)
        for id, agent in enumerate(agents):
            fitness[id] = obj_function(agent, train_X, val_X, train_Y, val_Y, weight_acc)
            
        idx = np.argsort(-fitness)
        sorted_agents = agents[idx].copy()
        sorted_fitness = fitness[idx].copy()
        
    return sorted_agents, sorted_fitness

In [12]:
def compute_fitness(agent, train_X, test_X, train_Y, test_Y, weight_acc=0.99):
    # compute a basic fitness measure
    if(weight_acc == None):
        weight_acc = 0.99
    
    weight_feat = 1 - weight_acc
    num_features = len(agent)
   
    acc = compute_accuracy(agent, train_X, test_X, train_Y, test_Y)
    feat = (num_features - np.sum(agent))/num_features

    fitness = weight_acc * acc + weight_feat * feat
    
    return fitness

In [13]:
from sklearn.feature_selection import f_regression

# inputs:
#    X: pandas.DataFrame, features
#    yy: pandas.Series, target variable
#    K: number of features to select


X=x
yy=df[str].to_numpy()
K=x.shape[1]
print(yy[0])
print(X.shape)

# compute F-statistics and initialize correlation matrix
F = pd.Series(f_regression(X, yy)[0], index = X.columns)
corr = pd.DataFrame(.00001, index = X.columns, columns = X.columns)

# initialize list of selected features and list of excluded features
selected = []
not_selected = X.columns.to_list()
Dict={}
# repeat K times
for i in range(K):
  
    # compute (absolute) correlations between the last selected feature and all the (currently) excluded features
    if i > 0:
        last_selected = selected[-1]
        corr.loc[not_selected, last_selected] = X[not_selected].corrwith(X[last_selected]).abs().clip(.00001)
        
    # compute FCQ score for all the (currently) excluded features (this is Formula 2)
    score = F.loc[not_selected] / corr.loc[not_selected, selected].mean(axis = 1).fillna(.00001)
    
    # find best feature, add it to selected and remove it from not_selected
    # print(score[score.argmax()])
    # print(score.max())
    # print(score.index[score.argmax()])
    best = score.index[score.argmax()]
    selected.append(best)
    not_selected.remove(best)
    Dict[best]=score.max()

mrMr=[]
for i in range(x.shape[1]):
  mrMr.append(Dict[x.columns[i]])
#print(mrMr)  

1
(9670, 4005)


/tmp/ipykernel_32/3757669169.py:29: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  corr.loc[not_selected, last_selected] = X[not_selected].corrwith(X[last_selected]).abs().clip(.00001)
/tmp/ipykernel_32/3757669169.py:29: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  corr.loc[not_selected, last_selected] = X[not_selected].corrwith(X[last_selected]).abs().clip(.00001)
/tmp/ipykernel_32/3757669169.py:29: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  corr.loc[not_selected, last_selected] = X[not_selected].corrwith(X[last_selected]).abs().clip(.00001)
/tmp/ipykernel_32/3757669169.py:29: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when p

In [14]:
for i in range(len(mrMr)):
  mrMr[i]=mrMr[i]/(max(mrMr))

In [15]:
def normalize(arr, t_min, t_max):
    norm_arr = []
    diff = t_max - t_min
    diff_arr = max(arr) - min(arr)
    for i in arr:
        temp = (((i - min(arr))*diff)/diff_arr) + t_min
        norm_arr.append(temp)
    return norm_arr

In [16]:
!pip install Py-FS
def fitness_score_values(data,target):
    from Py_FS.filter import PCC as PCC
    from Py_FS.filter import MI as MI
    from sklearn.feature_selection import mutual_info_classif
    from Py_FS.filter import Relief as Relief
    from sklearn.feature_selection import chi2
    
    print(data)
    print(target)
    #print('Pcc scores')
    #PCC_solution = PCC(data, target)
    #print(PCC_solution.scores)
    print('MI scores')
    mi_scores=mutual_info_classif(data, target, discrete_features='auto', n_neighbors=3, copy=True, random_state=None)
    print((mi_scores))
    
    print('Relief scores')
    Relief_solution = Relief(data, target)
    print((Relief_solution.scores))
    
   
    shapley=[]
    b=x.shape[1]
    print(b)
    for i in range(b):
        shapley.append((mi_scores[i]+mrMr[i]+Relief_solution.scores[i])/3)
    return shapley

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 52.2 MB/s eta 0:00:0000:0100:01


In [17]:
target=df[str].to_numpy()
print(len(target), target)

9670 [1 1 1 ... 0 0 0]


In [18]:
!pip install tabulate
!pip install ReliefF
score=fitness_score_values(x.values,target)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.1/48.1 kB 1.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for ReliefF: filename=ReliefF-0.1.2-py3-none-any.whl size=6319 sha256=ea298d2bdca40cd5c16e295de0670414b33d7552156bc8e192205ba561a7b6a7
  Stored in directory: /root/.cache/pip/wheels/35/bc/0b/14a0227d4829b6b27f25de250213eb348b094d075be7604707
Successfully built ReliefF
[[0.01891764 0.33715841 0.18103912 ... 0.00618417 0.06937568 0.02094024]
 [0.01134355 0.33530885 0.14427616 ... 0.00686109 0.18043566 0.15891416]
 [0.04868682 0.25834095 0.17169254 ... 0.         0.00734931 0.00738714]
 ...
 [0.         0.04179414 0.10518043 ... 0.         0.         0.        ]
 [0.00071266 0.08072688 0.25900993 ... 0.         0.00753672 0.        ]
 [0.00523137 0.05519465 0.06984088 ... 0.         0.         0.        ]]
[1 1 1 ... 0 0 0]
MI scores
[0.23643302 0.28411081 0.10715336 ... 0.34885485 0.36316147 0.36549842]
Relief scores
[0.00819399 0.000417   0.         ..

In [25]:
score = np.asarray(score)
print(score.shape)

(4005,)


In [26]:
# Returns the rank array
def ranking(score):
    rank = np.copy(score)
    rank_x = np.sort(score)
    for i in range(len(rank_x)):
        for j in range(len(rank_x)):
            if rank_x[i] == score[j]:
                rank[j] = i
    return np.asarray(rank) #rank

#Takes the ranking array consisting of rank in the positions and creates the 0th iteration feature map
def random_feature(rank, org_feat):
    feat_0 = np.copy(org_feat)
    cutoff = 0.5*(len(rank))
    for i in range(len(rank)):
        if rank[i] < cutoff:
            feat_0[i] = org_feat[i]
        else:
            feat_0[i] = 0
    return feat_0 #0th iteration feature map

#Mutates by deleting selected feature and replacing by non selected ones
def randomize(rank,feat_0,org_feat):
    feat_1 = np.copy(feat_0)
    replace = 0.1*(len(rank)) #count of number of mutation
    for i in range(int(replace)): #masking random selected top feats 
        index = np.random.randint(0.5*(len(rank))) #random index for mutation
        for k in range(len(rank)):
            if rank[k] == index:
                feat_1[index] = 0
    for j in range(len(rank)-int(replace),len(rank)): #top 10% unselected elements are selected
        feat_1[j] = org_feat[j]
    return feat_1 #2nd,3rd,.... iteration feature map

def mask(feat_1):
    mask = np.copy(feat_1)
    for i in range(len(feat_1)):
        if feat_1[i]>0:
            mask[i]=1
        else:
            mask[i]=0
    return mask

In [27]:
omega=0.9
def compute_accuracy(solution, trainX, trainy, testX,testy):
    cols=np.flatnonzero(solution)
    val=1
    if np.shape(cols)[0]==0:
        return val
    clf=KNeighborsClassifier(n_neighbors=7)
    train_data=trainX[:,cols]
    test_data=testX[:,cols]
    clf.fit(train_data,trainy)
    error=1-clf.score(test_data,testy)

    #in case of multi objective  []
    featureRatio = (solution.sum()/np.shape(solution)[0])
    val=omega*error+(1-omega)*featureRatio
    # print(error,featureRatio,val)
    return val 

In [28]:
(a,b)=np.shape(df)
trainX = df.values[:noTrainRows,0:b-1]
testX  = df.values[noTrainRows:,0:b-1]
trainy = target[0:noTrainRows]
testy  = target[noTrainRows:]

In [30]:
rank=ranking(score)
feat_prev=random_feature(rank,x.values[0])
feat_next=randomize(rank,feat_prev,x.values[0])
best=feat_prev
n=1000 #no. of iterations
for i in range (1,n+1):
    print("Iteration:")
    print(i)
    feat=compute_accuracy(feat_prev,trainX,trainy,testX,testy)
    feat1=compute_accuracy(feat_next,trainX,trainy,testX,testy)
    if feat1<feat:
        best=feat_next
        feat_prev=feat_next
    print("Accuracy:")
    print(1-(compute_accuracy(best,trainX,trainy,testX,testy)))
    
    feat_next=randomize(rank,feat_prev,x.values[0])

Iteration:
1
Accuracy:
0.9592892154092021
Iteration:
2
Accuracy:
0.9599120256071734
Iteration:
3
Accuracy:
0.9621643929354998
Iteration:
4
Accuracy:
0.9625222776807729
Iteration:
5
Accuracy:
0.9646746250037673
Iteration:
6
Accuracy:
0.9667595573735799
Iteration:
7
Accuracy:
0.9670169977368578
Iteration:
8
Accuracy:
0.9689721806432863
Iteration:
9
Accuracy:
0.9691481577264605
Iteration:
10
Accuracy:
0.9692831160674563
Iteration:
11
Accuracy:
0.9710923250679121
Iteration:
12
Accuracy:
0.9729106244567767
Iteration:
13
Accuracy:
0.9729106244567767
Iteration:
14
Accuracy:
0.9729106244567767
Iteration:
15
Accuracy:
0.9748156404292422
Iteration:
16
Accuracy:
0.9748156404292422
Iteration:
17
Accuracy:
0.9748767560399596
Iteration:
18
Accuracy:
0.9748767560399596
Iteration:
19
Accuracy:
0.9748767560399596
Iteration:
20
Accuracy:
0.9748767560399596
Iteration:
21
Accuracy:
0.9748767560399596
Iteration:
22
Accuracy:
0.9748767560399596
Iteration:
23
Accuracy:
0.9748767560399596
Iteration:
24
Accura

In [31]:
masked = mask(best)
print(masked)
count=0
for i in range(len(masked)):
    if masked[i]>0:
        count=count+1
print(count)

[0. 0. 0. ... 1. 1. 1.]
410
